## MWE (Understanding the Ensemble Tools)

Goal of this section is to adapt a vanilla example from the tutorial with our reduced framework (i.e., only store moment information.)

Overview:

- In the output_func, it gets the solution for a particular path (and that trajectory's index `i`), and gets to transform it to feed to the `reduction_func`.

- In the reduction, it gets the `u` (which ends up being our `sim.u`, which we transform as we go along.) And the `data`, which is an array of all the various `output_func` trajectory outputs. And `I`, which I guess is a list/indexing set of trajectories.

### General Setup

In [1]:
using DifferentialEquations

In [2]:
using Statistics, StatsBase

In [6]:
function output_func(sol,i)
  return summarystats(sol.u), false # for a given solution, just return the summary statistics
end

output_func (generic function with 1 method)

In [7]:
# boilerplate from the tutorial
prob = ODEProblem((u,p,t)->1.01u,0.5,(0.0,1.0))

function prob_func(prob,i,repeat)
  ODEProblem(prob.f,rand()*prob.u0,prob.tspan)
end

prob_func (generic function with 1 method)

In [8]:
function reduction(u,batch,I)
  # return summary statistics on the mean and minimum (for ex.)
  means = [sol.mean for sol in batch]
  mins = [sol.min for sol in batch] 
  newstuff = (summarystats(means), summarystats(mins))
  append!(u, newstuff)
  return u, false
end

reduction (generic function with 1 method)

In [9]:
prob2 = EnsembleProblem(prob,prob_func=prob_func,output_func=output_func,reduction=reduction,u_init=0.0)

EnsembleProblem with problem ODEProblem

In [ ]:
sim2 = solve(prob2,Tsit5(),trajectories=100,batch_size=20)